This notebook is WIP, do not use.
We haven't found a way to load the model trained on OpenAI quick sample data to evaluate on the curated holdout data.

# Set up

In [1]:
from datasets import load_dataset
from transformers import pipeline
import torch

# Load data

In [2]:
raw_datasets = load_dataset("dvquys/restaurant-reviews-public-sources")

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'Comments', 'tokens', 'ner_tags'],
        num_rows: 1590
    })
    val: Dataset({
        features: ['id', 'text', 'Comments', 'tokens', 'ner_tags'],
        num_rows: 398
    })
    test: Dataset({
        features: ['id', 'text', 'Comments', 'tokens', 'ner_tags'],
        num_rows: 10
    })
})

In [5]:
raw_datasets["test"][0]["tokens"]

['The',
 'restaurant',
 'looks',
 'out',
 'over',
 'beautiful',
 'green',
 'lawns',
 'to',
 'the',
 'Hudson',
 'River',
 'and',
 'the',
 'Statue',
 'of',
 'Liberty',
 '.']

In [6]:
raw_datasets["test"][0]["ner_tags"]

[0, 0, 19, 20, 20, 20, 20, 20, 0, 0, 19, 20, 20, 20, 20, 20, 20, 0]

# Load model

In [3]:
# Local model
output_dir = 'bert-finetuned-ner-accelerate'
token_classifier = pipeline(
    "token-classification", model=output_dir, aggregation_strategy="simple", device='cuda'
)
token_classifier('The place was nice and calm.')

[{'entity_group': 'FOOD',
  'score': 0.14983541,
  'word': 'The',
  'start': 0,
  'end': 3}]

In [4]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    output_dir
)

# Metrics

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [6]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [7]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs


In [8]:
tokenized_datasets = test_dataset.map(
    tokenize_and_align_labels,
    batched=True
)

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

Map:   0%|          | 0/398 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorForTokenClassification
from torch.utils.data import DataLoader

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

test_dataloader = DataLoader(
    tokenized_datasets['val'], collate_fn=data_collator, batch_size=8
)

In [11]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`id` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [10]:
for batch in test_dataloader:
    print(batch)

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`id` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [29]:
for batch in test_dataloader:
    with torch.no_grad():
        outputs = model(**batch)

    predictions = outputs.logits.argmax(dim=-1)
    labels = batch["labels"]

    # Necessary to pad predictions and labels for being gathered
    predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

    predictions_gathered = accelerator.gather(predictions)
    labels_gathered = accelerator.gather(labels)

    true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
    metric.add_batch(predictions=true_predictions, references=true_labels)

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`id` in this case) have excessive nesting (inputs type `list` where type `int` is expected).